## Cluster Studies (Jan)

Here is an updated version of the ClusterStudies.ipynb notebook, using uproot4 and newer versions of our plotting utilities.

In [ ]:
#import libraries and some constants
import os, sys, pathlib
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import ROOT as rt # I will use this for some plotting
import uproot as ur
from numba import njit

path_prefix = os.getcwd() + '/../'
modelpath = path_prefix+'clusters/Models/'

if(path_prefix not in sys.path): sys.path.append(path_prefix)
from util import resolution_util as ru
from util import plot_util as pu
from util import ml_util as mu
from util import qol_util as qu

# metadata
layers = ["EMB1", "EMB2", "EMB3", "TileBar0", "TileBar1", "TileBar2"]
cell_size_phi = [0.098, 0.0245, 0.0245, 0.1, 0.1, 0.1]
cell_size_eta = [0.0031, 0.025, 0.05, 0.1, 0.1, 0.2]
len_phi = [4, 16, 16, 4, 4, 4]
len_eta = [128, 16, 8, 4, 4, 2]
cell_shapes = {layers[i]:(len_eta[i],len_phi[i]) for i in range(len(layers))}
cell_widths = {layers[i]:(cell_size_eta[i],cell_size_phi[i]) for i in range(len(layers))}

In [ ]:
# fancy display names for each pion type
pi_latex = {
    'p0': '#pi^{0}',
    'pp': '#pi^{#pm}',
}

pi_latex_plt = {
    'p0': '$\pi^{0}$',
    'pp': '$\pi^{\pm}$'
}

# Plotting settings
# xkcd -- turn this on for fun-looking (but marginally less useful) plots
use_xkcd = False
if(use_xkcd):
    mode = 'light'
    plt.xkcd(scale=.75,length=100,randomness=1)
    
# plotting style -- manages our color palette and object colors
mode = 'dark' # for publications, use "light"
plotstyle = qu.PlotStyle(mode)
plotstyle.SetStyle()
    
# some matplotlib-specific stuff
params = {'legend.fontsize': 13,
          'axes.labelsize': 16,
          'axes.titlesize': 18
         }
plt.rcParams.update(params)

Let's fetch the data we want to look at. We'll need to change the code below depending on what dataset we're inspecting (and however you've stored it on your system).

In [ ]:
inputpath = path_prefix+'data/pion_converted/'
drpath = os.getcwd() + '/dr/pion_converted/'

datafiles = {
    'p0':inputpath + 'user.angerami.mc16_13TeV.900246.PG_singlepi0_logE0p2to2000.e8312_e7400_s3170_r12383.v01-45-gaa27bcb_OutputStream/*.root',
    'pp':inputpath + 'user.angerami.mc16_13TeV.900247.PG_singlepion_logE0p2to2000.e8312_e7400_s3170_r12383.v01-45-gaa27bcb_OutputStream/*.root'
}

drfiles = {key: val.replace(inputpath, drpath) for key,val in datafiles.items()}

cluster_branches = [
    'cluster_E', 'cluster_E_LCCalib', 
    'cluster_Pt', 'cluster_Eta', 'cluster_Phi', 
    'cluster_nCells',
    'cluster_ENG_CALIB_TOT', 'cluster_EM_PROBABILITY',
]


dr_branches = [
    'cluster_E', 'cluster_Eta', 'cluster_Phi', 'cluster_ENG_CALIB_TOT',
    'truth_E', 'truth_Eta', 'truth_Phi', 'truth_PdgId',
    'dR'
]

event_branches = [
    'truthPartPt',
    'truthPartE',
    'truthPartEta',
    'truthPartPhi',
    'nCluster',
    'clusterCount'
]

colors = {
    'pp':rt.kRed,
    'p0':plotstyle.curve
}

keys = list(datafiles.keys())

In [ ]:
# TODO: branch_filter is not working

cluster_tree = {
    key:ur.lazy(':'.join((val,'ClusterTree')), branch_filter=lambda x: x.name in cluster_branches)
    for key,val in datafiles.items()
}

# We have some info on the minimum delta R between clusters and (stable) truth-level particles,
# tucked away in another set of files produced by our deltaR.py/deltaR.C script.
dr_tree = {
    key:ur.lazy(':'.join((val,'dr_info')), branch_filter=lambda x: x.name in dr_branches)
    for key,val in drfiles.items()
}

In [ ]:
# Perform some cuts on the data
indices = {
    key:np.full(len(val),True,dtype=bool)
    for key,val in cluster_tree.items()
}

cuts = {
    'eta':('cluster_Eta','window',(-0.7,0.7)),
    'energy':('cluster_ENG_CALIB_TOT','lower',0.2)
}

for cutname, cut in cuts.items():
    var, cuttype, cutval = cut
    
    for key in keys:
        if cuttype == 'lower': sel = cluster_tree[key][var] > cutval
        elif cuttype == 'upper': sel = cluster_tree[key][var] < cutval
        elif cuttype == 'window':sel = (cluster_tree[key][var] > cutval[0]) * (cluster_tree[key][var] < cutval[1])
        else: continue
        indices[key] *= sel.to_numpy()
        
cluster_tree = {key:val[indices[key]] for key,val in cluster_tree.items()}
dr_tree      = {key:val[indices[key]] for key,val in dr_tree.items()}

In [ ]:
# Sanity check that things line up
#assert(np.sum(cluster_tree['pp']['cluster_E'] - dr_tree['pp']['cluster_E']) == 0.)

In [ ]:
# Convenience functions for histogramming, from numpy to ROOT.
# Much faster than looping over data for sequential filling of histograms.
def hist1d(x,h):
    N = h.GetNbinsX()
    bin_i = h.GetBinLowEdge(1)
    bin_f = h.GetBinLowEdge(N) + h.GetBinWidth(N)    
    np_hist = np.histogram(x, bins=N, range=(bin_i, bin_f))[0]
    for i in range(N):
        h.SetBinContent(i+1, np_hist[i])
    return

def hist2d(x,y,h):
    
    Nx = h.GetNbinsX()
    Ny = h.GetNbinsY()
    
    bin_xi = h.GetXaxis().GetBinLowEdge(1)
    bin_xf = h.GetXaxis().GetBinLowEdge(Nx) + h.GetXaxis().GetBinWidth(Nx)
    
    bin_yi = h.GetYaxis().GetBinLowEdge(1)
    bin_yf = h.GetYaxis().GetBinLowEdge(Ny) + h.GetYaxis().GetBinWidth(Ny)
      
    np_hist = np.histogram2d(x,y, bins=(Nx,Ny), range=((bin_xi,bin_xf),(bin_yi, bin_yf)))[0]
    for i in range(Nx):
        for j in range(Ny):
            h.SetBinContent(i+1,j+1,np_hist[i,j])
    return

## Plotting reco vs truth energy

In [ ]:
#fig, axs = plt.subplots(1,2,figsize=(24,12))



## Plotting average cluster images

Let's plot average images for each calo layer, for $\pi^0$ and $\pi^\pm$.

Specifically, let's calculate the average of *normalized* images. This means that we normalize each cluster's set of images -- rescale them so the integral across calo layers is 1 -- and then average the results. This is very different than averaging and then normalizing, but given that our classifiers use normalized images (and perform the normalization on-the-fly) the average of norm'd images is meaningful.

Furthermore, we can compare this average to the average images in the old datasets (where normalization was applied).

To accomplish this, we're going to use some fancy `uproot` stuff, specifically its `iterate` functionality. This seems more practical than just using the lazy arrays we made earlier, as they're pretty big and doing reductions on them is quite slow/intensive.

In [ ]:
rfile = 'avg_images.root'
cell_threshold = None # TODO: Allow us to exclude cells below some threshold value

if(not pathlib.Path(rfile).exists()):

    f = rt.TFile(rfile,"RECREATE")
    
    step_size = 1000 # number of entries to read in each iteration
    loop_keys = layers + ['cluster_ENG_CALIB_TOT','cluster_Eta']

    # Numpy buffers for images
    avg_images = {
        key:{
            layer:np.zeros(cell_shapes[layer])
            for layer in layers
        }
        for key in keys
    }
    
    # Histograms for images
    im_hists = {
        key:{
            layer:rt.TH2F(
                qu.RN(),
                '{} ({});#eta;#phi'.format(layer, pi_latex[key]),
                cell_shapes[layer][0],
                -0.5 * cell_shapes[layer][0] * cell_widths[layer][0],
                 0.5 * cell_shapes[layer][0] * cell_widths[layer][0],
                cell_shapes[layer][1],
                -0.5 * cell_shapes[layer][1] * cell_widths[layer][1],
                 0.5 * cell_shapes[layer][1] * cell_widths[layer][1]
            )
            for layer in layers
        }
        for key in keys
    }

    n = 0
    for key,val in datafiles.items():
        for array in ur.iterate(':'.join((val,'ClusterTree')), loop_keys, step_size=step_size):

            # Apply cuts
            #TODO: Unify this with cuts applied earlier? Or go back to the existing lazy arrays?
            indices = np.full(len(array),False,bool)

            sel = (array['cluster_ENG_CALIB_TOT'] > 0.2).to_numpy()
            sel *= (np.abs(array['cluster_Eta'].to_numpy()) < 0.7)
            array = array[sel]
            n += len(array)

            ims = {layer: array[layer].to_numpy() for layer in layers}
            norms = {layer: np.sum(x,axis=(1,2)) for layer,x in ims.items()}
            norms = np.sum([x for x in norms.values()],axis=0)
            norms[norms == 0.] = 1.
            ims = {layer: np.divide(ims[layer], norms.reshape(-1,1,1)) for layer in ims.keys()}

            for layer in layers:
                avg_images[key][layer] += np.sum(ims[layer],axis=0)    
                
        for layer in layers:
            avg_images[key][layer] /= n
            
            for i in range(cell_shapes[layer][0]):
                for j in range(cell_shapes[layer][1]):
                    im_hists[key][layer].SetBinContent(i+1,j+1,avg_images[key][layer][i,j])
            
            im_hists[key][layer].Write('{}_{}'.format(key,layer))            
    f.Close()
            
f = rt.TFile(rfile,"READ")
    # Histograms for images
im_hists = {
    key:{
        layer: f.Get('{}_{}'.format(key,layer))
        for layer in layers
    }
    for key in keys
}

for key in keys:
    for layer in layers:
        im_hists[key][layer].SetDirectory(0)
f.Close()

In [ ]:
rt.gStyle.SetOptStat(0)
c = rt.TCanvas(qu.RN(),qu.RN(),2400,2400)
c.Divide(3,4)
i = 0
for key in keys:
    for layer in layers:
        c.cd(i+1)
        im_hists[key][layer].Draw('COLZ')
        i += 1
c.Draw()
c.SaveAs('avg_ims.png')

## Plotting Cluster and cell energies

Let's look at the energies of individual cells, i.e. pixels in our calorimeter images.

Note that as our dataset has *many* clusters, and each cluster has *many* pixels (in multi-dim awkward arrays), preparing the histograms is very slow for the full dataset. To deal with this, we will load the histogram from a ROOT file if it exists -- so that we only have to compute the histograms once.

Another potential solution would be to do this plotting in ROOT/C++, much like how we handle the computation of $\Delta R$. But keeping this in Python and dealing with a little slowness might be easier.

We'll also display the histograms using `matplotlib` so that we can use the "symlog" axis.

In [ ]:
rfile = 'cell_energies.root'

if(not pathlib.Path(rfile).exists()):
    # rng = np.random.default_rng()
    # suppression_factor = 1000
    h1 = {
        layer:{
            key:rt.TH1F(qu.RN(),'{} ({});Energy;% Count'.format(layer,key), int(8e4),-40.,40.)
            for key in keys
        }
        for layer in layers
    }

    for key in keys:
        n = len(cluster_tree[key])
    #     idx = np.arange(n)
        msk = np.full(n,True,dtype=bool)
    #     idx = rng.choice(idx,int(n/suppression_factor),replace=False)
    #     msk[idx] = True
    #     print(np.sum(msk))
        for layer in layers:
            hist1d(cluster_tree[key][layer][msk].to_numpy().flatten(), h1[layer][key])
            h1[layer][key].Scale(1./h1[layer][key].Integral())

    f = rt.TFile(rfile,'RECREATE')
    for layer in layers:
        for key in keys:
            f.cd()
            h1[layer][key].Write('{}_{}'.format(key,layer))
    f.Close()

f = rt.TFile(rfile,'READ')
h1 = {}
for layer in layers:
    h1[layer] = {}
    for key in keys:
        h1[layer][key] = f.Get('{}_{}'.format(key,layer))
        h1[layer][key].SetDirectory(0) # copy to memory, so that we can close our ROOT file
f.Close()

In [ ]:
def Hist2XY(h):
    N = h.GetNbinsX()
    bin_i = h.GetBinLowEdge(1)
    bin_f = h.GetBinLowEdge(N) + h.GetBinWidth(N)
    
    x_vals = np.linspace(bin_i, bin_f, N)
    y_vals = np.array([h.GetBinContent(i+1) for i in range(N)])
    return x_vals, y_vals

In [ ]:
fig, axs = plt.subplots(2,3,figsize=(24,12))

for i,key in enumerate(keys):
    for j,layer in enumerate(layers):
        ax = axs.flatten()[j]
        x,y = Hist2XY(h1[layer][key])
        ax.plot(x,y,label=pi_latex_plt[key])

for i,ax in enumerate(axs.flatten()):
    ax.set_xscale('symlog', linthresh=2.0e-2)
    ax.set_yscale('log')
    
    ax.set_xlim(-1.0e-2,1.0e2)
    
    ax.set_title(layers[i])
    ax.set_xlabel('Cell Energy [GeV]')
    ax.set_ylabel('% Count')
    
    ax.legend()
    plotstyle.SetStylePlt(ax)

fig.savefig('cell_energies.png')

## Plotting $\Delta R$ (versus Energy)

In [ ]:
# 1D histograms
h1 = {
    'dR':{
        key:rt.TH1F(qu.RN(),'{};#Delta R;% Count'.format(pi_latex[key]),120,0.,6.) 
        for key in keys
    },
    'cluster_E':{
        key:rt.TH1F(qu.RN(),'{};E;% Count'.format(pi_latex[key]),2500,0.,2500.) 
        for key in keys
    }
}


h2 = {
    key:rt.TH2F(qu.RN(),'{};#Delta R;Energy [GeV];% Count'.format(pi_latex[key]),240,0.,6., 2500,0.,2500.) 
    for key in keys
}

for key in keys:
    n = len(cluster_tree[key])
    hist1d(dr_tree[key]['dR'].to_numpy(), h1['dR'][key])
    hist1d(cluster_tree[key]['cluster_E'].to_numpy(), h1['cluster_E'][key])
    
    hist2d(
        dr_tree[key]['dR'].to_numpy(), 
        cluster_tree[key]['cluster_E'].to_numpy(), 
        h2[key]
    )
    
    for h in h1.values():
        h[key].Scale(1./h[key].Integral())

In [ ]:
rt.gStyle.SetOptStat(0)
c1 = rt.TCanvas(qu.RN(),'c1',1600,600)
c2 = rt.TCanvas(qu.RN(),'c2',1600,600)
c1.Divide(2,1)
c2.Divide(2,1)
margin = 0.2

c1.cd(1)
hstack_a = rt.THStack()
legend = rt.TLegend(0.7,0.7,0.9,0.9)
legend.SetTextColor(plotstyle.text)
for key in keys: 
    h1['dR'][key].SetLineColor(colors[key])
    hstack_a.Add(h1['dR'][key])
    legend.AddEntry(h1['dR'][key],pi_latex[key],'l')
hstack_a.SetTitle('Min. #Delta R (truth, cluster);#Delta R;% Count')
hstack_a.Draw('NOSTACK HIST')
legend.Draw()
rt.gPad.SetLogx()
rt.gPad.SetLogy()

c1.cd(2)
hstack_b = rt.THStack()
for key in keys: 
    h1['cluster_E'][key].SetLineColor(colors[key])
    hstack_b.Add(h1['cluster_E'][key])
hstack_b.SetTitle('cluster_E;E [GeV];% Count')
hstack_b.Draw('NOSTACK HIST')
legend.Draw()
rt.gPad.SetLogx()
rt.gPad.SetLogy()

c2.cd()
for i,key in enumerate(keys):
    c2.cd(i+1)
    rt.gPad.SetRightMargin(margin)
    h2[key].Draw('COLZ')
    #colorbar = h2[key].GetListOfFunctions().FindObject('palette')
    #colorbar.SetX1NDC(1.- 0.9 * margin)
    #colorbar.SetX2NDC(1.- 0.7 * margin)
    rt.gPad.SetLogx()
    rt.gPad.SetLogy()
    rt.gPad.SetLogz()
    
c1.Draw()
c2.Draw()

c1.SaveAs('dr.png')
c2.SaveAs('dr_e.png')

Let's also look at the PDG Id's of the nearest truth particles.

In [ ]:
pdg_codes = {
    11 : 'e^{#pm}',
    22 : '#gamma',
    111 : '#pi^{0}',
    211: '#pi^{#pm}'
}

pdg_vals = {
    11: 0,
    22: 1,
    111: 2,
    211: 3
}

pdg_bar_chart = {
    key: rt.TH1F(qu.RN(),'Truth match;Particle;% Count',4,0,4)
    for key in keys
}

for key,chart in pdg_bar_chart.items():
    vals = np.abs(dr_tree[key]['truth_PdgId'].to_numpy())
    vals = np.array([pdg_vals[x] for x in vals])
    hist1d(vals,chart)
    chart.Scale(1./chart.Integral())
    

In [ ]:
c = rt.TCanvas(qu.RN(),'c',800,600)
legend = rt.TLegend(0.8,0.8,0.9,0.9)
legend.SetTextColor(plotstyle.text)
stack = rt.THStack()
stack.SetTitle('Truth match;Particle;% Count')
for key,chart in pdg_bar_chart.items(): 
    chart.SetLineColor(colors[key])
    chart.SetFillColorAlpha(colors[key],0.3)
    stack.Add(chart)
    legend.AddEntry(chart,pi_latex[key],'lf')

stack.Draw('NOSTACK HIST')
legend.Draw()
rt.gPad.SetLogy()
stack.SetMaximum(3. * stack.GetMaximum())
rt.gPad.SetGrid()

stack.GetXaxis().SetNdivisions(-4)
stack.GetXaxis().SetLabelSize(0.05)
for i,code in enumerate(pdg_codes.values()):
    stack.GetXaxis().SetBinLabel(i+1,code)

c.Draw()
c.SaveAs('particle_id.png')

We see that for charged pions, the truth particles are always the truth particles themselves -- this makes sense (the truth particles probably come from the generator level, *before* the hadronic showering).

For the neutral pions, we have a mix of gammas, electrons/positrons and neutral pions themselves. The gammas completely dominate.